In [4]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder


In [5]:
pip install chardet


In [6]:
import pandas as pd
import chardet

# Detect the encoding
file_path = '/content/drive/MyDrive/IMDb Movies India.csv'
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

# Read the file with the detected encoding
df = pd.read_csv(file_path, encoding=encoding)


In [7]:
# Show data
df.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [8]:
# Check null values
df.isna().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [10]:
mean_values = df.select_dtypes(include=[np.number]).mean()

# Replace missing values in numeric columns with the mean
df_filled = df.fillna(mean_values)

In [11]:
df.isna().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [12]:
# Convert 'Votes' to string and remove non-digit characters
df['Votes'] = df['Votes'].astype(str).str.replace(r'\D', '', regex=True)



In [13]:
# Convert to numeric, coercing errors to NaN
df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')



In [14]:
# Replace NaN values with the mean of the 'Votes' column
df['Votes'].fillna(df['Votes'].mean(), inplace=True)



In [15]:
# Print the first few entries of 'Votes'
print(df['Votes'].head())

0    1938.340783
1       8.000000
2    1938.340783
3      35.000000
4    1938.340783
Name: Votes, dtype: float64


In [16]:
# Drop rows where 'Rating' column has NaN values
df_cleaned = df.dropna(subset=['Rating'])

In [17]:
# Extract numeric part of 'Duration' and convert to float
df['Duration'] = df['Duration'].str.extract('(\d+)').astype(float)

In [18]:
# Fill NaN values in 'Duration' with the mean of the column
mean_duration = df['Duration'].mean()  # Calculate the mean of 'Duration'
df['Duration'].fillna(mean_duration, inplace=True)  # Replace NaN with the mean

In [19]:
df_encoded = pd.get_dummies(df, columns=['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3'])


In [20]:
columns = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']


In [21]:
ohe = OneHotEncoder(drop="first", sparse=False, handle_unknown='ignore')
transformed_data = ohe.fit_transform(df[columns])
feature_names = ohe.get_feature_names_out(columns)
transformed_df = pd.DataFrame(transformed_data, columns=feature_names)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [22]:
# Droping original categorical columns and concatenate transformed DataFrame
df = df.drop(columns, axis=1)
df = pd.concat([df, transformed_df], axis=1)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna(0)


In [23]:
X=df.drop(columns=["Rating"])
y=df["Rating"]

In [24]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [25]:
# Initialize the Random Forest regression model
model = RandomForestRegressor()




In [ ]:
# Train the model using the training sets
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)



Mean Squared Error: 0.7632957612651338
R^2 Score: 0.9184367691025724
